In [1]:
instructions = """You are a resume-parsing assistant. Given the resume extract the data and return only valid JSON.
    Your Task is to just do categorization of the data, do not change the original text. Do not add any extra fields data to some other field.
    1)do not give me "\n" in the result
    2) Do not change the orignal text only just extract the data from the resume and return it in JSON format.
    The JSON must be in this format:
    {
      "name": "...",
      "email": "...",
      "phone": "...",
      "skills": ["..."],
      "education": ["..."],
      "experience": ["..."],
      "certifications": ["..."],
      "achievements": ["..."],
      "tech_stack": ["..."]
      "resume_id":["..."]
      "other_info": ["..."]
    }"""

In [2]:
from dotenv import load_dotenv
load_dotenv()
import nest_asyncio
import os
from llama_parse import LlamaParse
import json
nest_asyncio.apply()
import json
from sentence_transformers import SentenceTransformer
import pinecone
import os



e:\Hackathon\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:



parser1 = LlamaParse(result_type="json",verbose=True, premium_mode=True, api_key=os.environ.get("LLAMA_API_KEY") ,parsing_instruction=instructions)

In [ ]:
import os
import json

###########################
# MERGE & PARSE FUNCTIONS #
###########################

def merge_page_data(merged_resume, page_data):
    """
    Merge data from `page_data` into `merged_resume`.
    - If the field is a list, extend it (e.g., skills, experience).
    - If the field is a string, only overwrite if the new string is non-empty.
    - For everything else, do a simple overwrite.
    """
    for key, value in page_data.items():
        if key not in merged_resume:
            # If it's a brand new key, just add it
            merged_resume[key] = value
        else:
            # If it’s a list, extend the existing list
            if isinstance(value, list):
                merged_resume[key].extend(value)
            # If it’s a string, only overwrite if merged is empty or new one is non-empty
            elif isinstance(value, str):
                if not merged_resume[key] and value:
                    merged_resume[key] = value
            else:
                # For any other data type, just overwrite
                merged_resume[key] = value
    return merged_resume

def parse_multi_page_resume(parser, pdf_path):
    json_result = parser.get_json_result(pdf_path)

    # Initialize a "blank" structure that matches the final JSON layout
    combined_resume = {
        "name": "",
        "email": "",
        "phone": "",
        "skills": [],
        "education": [],
        "experience": [],
        "certifications": [],
        "achievements": [],
        "tech_stack": [],
        "resume_id": [],
        "other_info": []
    }
    
    # Go through each page and merge
    for page in json_result[0]['pages']:
        page_md = page['md']  # The JSON string in the 'md' key
        page_dict = json.loads(page_md)
        combined_resume = merge_page_data(combined_resume, page_dict)
    
    return combined_resume




In [3]:
# Directory containing resumes
resume_dir =os.environ.get("RESUME_DIR")
output_json_path = os.environ.get("PARSED_JSON")


In [ ]:
###################
# MAIN PIPELINE   #
###################


# Load existing parsed resumes (if file exists)
if os.path.exists(output_json_path):
    with open(output_json_path, 'r', encoding='utf-8') as f:
        existing_resumes = json.load(f)
else:
    existing_resumes = []

# Extract existing resume IDs to avoid duplicates
existing_ids = {json.dumps(r.get("resume_id")) for r in existing_resumes}

# Collect all PDF paths
pdf_files = [
    os.path.join(resume_dir, f)
    for f in os.listdir(resume_dir)
    if f.endswith('.pdf')
]

parsed_resumes = []

# Loop through all PDFs
for pdf_file in pdf_files:
    try:
        merged_resume_dict = parse_multi_page_resume(parser1, pdf_file)
        resume_id = json.dumps(merged_resume_dict.get("resume_id"))
        
        if resume_id not in existing_ids:
            parsed_resumes.append(merged_resume_dict)
            existing_ids.add(resume_id)
        else:
            print(f"⚠️ Resume with ID {resume_id} already exists. Skipping.")
            
    except Exception as e:
        print(f"❌ Error parsing {pdf_file}: {e}")

# Combine existing + new resumes
all_resumes = existing_resumes + parsed_resumes

# Save to JSON
with open(output_json_path, 'w', encoding='utf-8') as f:
    json.dump(all_resumes, f, ensure_ascii=False, indent=4)

print(f"✅ Added {len(parsed_resumes)} new resumes. Total saved: {len(all_resumes)}")

Started parsing the file under job_id e87d90a5-0974-42a2-8e24-d7f011e47d37
Started parsing the file under job_id de4a2274-f13d-46af-b5ca-5d19fc7906bd
Started parsing the file under job_id fe00345a-6880-4545-8407-3c16c9fbf402
Started parsing the file under job_id e951e49a-20e1-4538-8e26-9735d1d40198
Started parsing the file under job_id 3ae58ca0-3452-431a-af13-f8b34327d76e
Started parsing the file under job_id c0b357e0-7560-49b0-a407-4b7400073e34
Started parsing the file under job_id 97d8de83-03f7-4a63-af3d-f34a9b0f36f6
Started parsing the file under job_id fa23af50-a654-4a6a-a4db-5da37009ce61
Started parsing the file under job_id 7469fb32-1271-49ac-89ef-551795e62f0a
Started parsing the file under job_id cbe4f94a-9a78-483c-9c63-84de9737821b
Started parsing the file under job_id 2bf5468a-f13e-4da4-ad9d-73a44660c10d
Started parsing the file under job_id aed61336-0dff-49fa-a9a1-abbb3d20a8d6
Started parsing the file under job_id 74591185-772c-428a-972f-3c75253cbe36
Started parsing the file 

In [4]:
# 1. Load parsed resumes from JSON
import json
with open(output_json_path, 'r', encoding='utf-8') as f:
    parsed_resumes = json.load(f)


In [5]:
len(parsed_resumes)

200

In [6]:

# 2. Fields to remove
fields_to_remove = {'name', 'email', 'phone', 'resume_id'}

def filter_resume(resume):
    return {k: v for k, v in resume.items() if k.lower() not in fields_to_remove}

def resume_to_text(resume_dict):
    return " ".join([str(v) for v in resume_dict.values()])


In [7]:

# 3. Initialize embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

In [8]:
import os
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(
    api_key=os.environ.get("PINECONE_API_KEY")
)
index_name = 'resume'
# Now do stuff
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name, 
        dimension=1536, 
        metric='euclidean',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-west-2'
        )
    )

In [9]:
pc.list_indexes().names()

['resume']

In [10]:
index = pc.Index(index_name)


In [11]:
from json_to_sqlite import ResumeDatabase
db = ResumeDatabase()

In [12]:

# 5. Insert resumes into Pinecone
for i, resume in enumerate(parsed_resumes):
    try:
        print("Resume:", resume)
        
        # Insert into SQL and get back the SQL id
        sql_id = db.insert_resume(resume)  # Assuming this returns a unique integer or string ID
        print("SQL ID:", sql_id)

        # Filter out PII
        clean_resume = filter_resume(resume)

        # Add SQL ID to metadata
        clean_resume["sql_id"] = sql_id


        # Convert to text and generate embedding
        text = resume_to_text(clean_resume)
        embedding = model.encode(text).tolist()
        print("embedding",embedding)

        # Insert into Pinecone with SQL ID in metadata
        index.upsert([
            (f"resume-{i}", embedding, clean_resume)
        ])

        print(f"✅ Inserted resume-{i} with SQL ID {sql_id}")
    except Exception as e:
        print(f"❌ Failed to insert resume-{i}: {e}")

Resume: {'name': 'Alyssa Chavez', 'email': 'alyssachavez88@gmail.com', 'phone': '+1-465-3587', 'skills': ['Cybersecurity - Skilled in penetration testing, risk assessment, and securing enterprise networks against cyber threats.'], 'education': ['Diploma in Software Engineering (2013-2015)', 'Hands-on experience in full-stack web development and mobile app creation.'], 'experience': ['Data Scientist at ABC Inc. (2019-2023)', 'Built predictive models that enhanced decision-making processes, reducing operational costs by 25%.'], 'certifications': ['AWS Certified Solutions Architect - Validated expertise in designing and deploying scalable AWS solutions, optimizing performance and security.'], 'achievements': ['Published a research paper on AI ethics - Contributed to an AI ethics framework adopted by industry leaders, shaping responsible AI development.'], 'tech_stack': ['Java', 'Spring Boot', 'MySQL', 'Kafka', 'Azure DevOps'], 'resume_id': ['C1061'], 'other_info': []}
SQL ID: 1
embedding 

In [ ]:
# # 3. Define the search function
# def search_resumes(job_description, top_k=5):
#     # Step 1: Create embedding from JD
#     jd_embedding = model.encode(job_description).tolist()

#     # Step 2: Query Pinecone
#     results = index.query(
#         vector=jd_embedding,
#         top_k=top_k,
#         include_metadata=True
#     )

#     # Step 3: Return matches
#     matches = []
#     for match in results.get("matches", []):
#         matches.append({
#             "score": match["score"],
#             "resume_id": match["id"],
#             "sql_id": match["metadata"].get("sql_id"),
#             "metadata": match["metadata"]
#         })
    
#     return matches




In [ ]:
# # 4. Example usage

# jd = """
# Description:
# We are looking for a skilled Machine Learning Engineer to develop, train, and deploy AI models for real-world applications. You will work with large datasets, optimize algorithms, and collaborate with cross-functional teams to drive innovation.

# Responsibilities:

# Develop and optimize machine learning models for various applications.
# Process and analyze large datasets to extract meaningful insights.
# Deploy and maintain AI models in production environments.
# Collaborate with data scientists, engineers, and product teams.
# Stay updated with the latest advancements in AI and ML.
# Qualifications:

# Bachelor’s or Master’s in Computer Science, Data Science, or a related field.
# Proficiency in Python, TensorFlow, PyTorch, and Scikit-learn.
# Experience with data preprocessing, model deployment, and cloud platforms.
# Strong problem-solving skills and analytical mindset.

# """
# top_matches = search_resumes(jd, top_k=5)

# for i, match in enumerate(top_matches):
#     print(f"\n🔍 Match #{i+1}")
#     print(f"🔗 Pinecone ID: {match['resume_id']}")
#     print(f"🗃 SQL ID: {match['sql_id']}")
#     print(f"📊 Score: {match['score']:.4f}")
#     print(f"📎 Metadata: {match['metadata']}")

In [ ]:
# top_matches

[{'score': 0.674601853,
  'resume_id': 'resume-137',
  'sql_id': 138.0,
  'metadata': {'achievements': ['Published a research paper on AI ethics - Contributed to an AI ethics framework adopted by industry leaders, shaping responsible AI development'],
   'certifications': ['AWS Certified Solutions Architect - Validated expertise in designing and deploying scalable AWS solutions, optimizing performance and security'],
   'education': ['Master of Business Administration (2017-2019) - Focused on Business Strategy, Financial Analysis, and Operations Management',
    'Diploma in Software Engineering (2013-2015) - Hands-on experience in full-stack web development and mobile app creation'],
   'experience': ['Data Scientist at ABC Inc. (2019-2023) - Built predictive models that enhanced decision-making processes, reducing operational costs by 25%'],
   'skills': ['Python',
    'Machine Learning',
    'TensorFlow',
    'PyTorch',
    'Scikit-learn',
    'AI solutions deployment'],
   'sql_id':